# Parkinson's Disease Classification with Hyperparameter Tuning

This notebook performs an end-to-end machine learning analysis to classify Parkinson's disease based on a set of biomedical voice measurements.

The process includes:
1.  **Data Loading and Preprocessing**: Fetching the dataset, splitting it, and scaling features.
2.  **Model Training**: Using `GridSearchCV` to find the best hyperparameters for several classification models, including a PyTorch-based Artificial Neural Network (ANN).
3.  **Evaluation**: Comparing models based on F1-score, accuracy, and other metrics.
4.  **Visualization**: Generating plots for model comparison, confusion matrices, ROC curves, and feature importance.
5.  **Interpretability**: Analyzing feature importances to understand which factors are most predictive.
6.  **Model Saving**: Saving the best-performing model and the feature scaler for future use.

In [1]:
# --- 1. Import Required Libraries ---
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import skorch
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.metrics import (
    classification_report, 
    f1_score, 
    accuracy_score, 
    precision_score, 
    recall_score, 
    roc_auc_score,
    confusion_matrix,
    roc_curve
)
from sklearn.inspection import permutation_importance

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib

# PyTorch and skorch for the Neural Network
import torch
import torch.nn as nn
from skorch import NeuralNetClassifier
from typing import Tuple

### PyTorch MLP Model Definition
This class defines a simple Multi-Layer Perceptron (MLP) using PyTorch. It includes `BatchNorm1d` and `Dropout` layers to help prevent overfitting, which is common in models with many parameters. The network is designed for tabular data.

In [2]:
class MLP(nn.Module):
    """Small MLP for tabular inputs with BatchNorm+Dropout to reduce overfitting."""
    def __init__(self, in_dim: int, hidden: Tuple[int, int] = (256, 128), dropout: float = 0.1):
        super().__init__()
        h1, h2 = hidden
        self.net = nn.Sequential(
            nn.Linear(in_dim, h1),
            nn.BatchNorm1d(h1),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(h1, h2),
            nn.BatchNorm1d(h2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(h2, 1)  # single logit
        )

    def forward(self, x):
        # skorch handles numpy-to-tensor conversion, but we ensure it's float32
        x = x.float() 
        return self.net(x).squeeze(1)

### 2. Setup and Data Preparation
- An output directory is created to store all results.
- The Parkinson's dataset is loaded from the UCI Machine Learning Repository.
- Features (`X`) and the target variable (`y`) are separated.
- The data is split into training and testing sets.
- `StandardScaler` is used to normalize the features, which is crucial for distance-based algorithms and neural networks.
- Data types are converted to `float32` to be compatible with PyTorch.

In [3]:
# Setup Output Directory
output_dir = 'parkinsons_analysis_results_gridsearch'
os.makedirs(output_dir, exist_ok=True)
print(f"All results and figures will be saved in the '{output_dir}' directory.")

# Data Loading and Preprocessing
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/parkinsons.data"
df = pd.read_csv(url)
X = df.drop(['name', 'status'], axis=1)
y = df['status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=99, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X.columns = df.drop(['name', 'status'], axis=1).columns

# Convert data types for PyTorch/skorch
X_train_scaled = X_train_scaled.astype(np.float32)
X_test_scaled = X_test_scaled.astype(np.float32)
y_train = y_train.values.astype(np.float32)
y_test = y_test.values.astype(np.float32)

All results and figures will be saved in the 'parkinsons_analysis_results_gridsearch' directory.


### 3. Define Parameter Grids for GridSearchCV
This dictionary contains the hyperparameter search spaces for each model. `GridSearchCV` will exhaustively test all combinations to find the best ones based on the F1-score. The `ANN (PyTorch)` grid includes parameters like learning rate, epochs, and network architecture (`module__hidden`).

In [4]:
param_grids = {
    "Logistic Regression": {
        'penalty': ['l1', 'l2'],
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'solver': ['liblinear', 'saga'],
        'class_weight': [None, 'balanced'],
        'max_iter': [4000]
    },
    "K-Nearest Neighbors": {
        'n_neighbors': list(range(1, 21)),
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    "Decision Tree": {
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 5, 7, 10],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'class_weight': [None, 'balanced']
    },
    "Support Vector Machine": {
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 0.1, 0.01, 0.001],
        'kernel': ['rbf', 'linear'],
        'probability': [True]
    },
    "Naive Bayes": {
        'var_smoothing': np.logspace(-9, -5, 10)
    },
    "Random Forest": {
        'n_estimators': [100, 250, 500],
        'max_depth': [None, 5, 10, 15],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2'],
        'class_weight': [None, 'balanced']
    },
    "XGBoost": {
        'n_estimators':    [100, 300, 500],
        'learning_rate':   [0.01, 0.05, 0.1],
        'max_depth':       [3, 5, 7],
        'min_child_weight':[1, 3, 5],
        'subsample':       [0.7, 0.9, 1.0],
        'colsample_bytree':[0.7, 0.9, 1.0]
    },
    "ANN (PyTorch)": {
        'lr': [1e-4, 1e-3, 1e-2],
        'max_epochs': [100, 300],
        'batch_size': [16, 32],
        'optimizer__weight_decay': [1e-5, 1e-4],
        'module__hidden': [(128, 64), (256, 128)],
        'module__dropout': [0.1, 0.2, 0.3]
    }
}

### 4. Perform GridSearchCV for Each Model
This is the core of the analysis. The code iterates through a list of models, including the `NeuralNetClassifier` from `skorch` which wraps our PyTorch `MLP`.

For each model:
1.  An instance is created. The PyTorch model is configured with early stopping to prevent overfitting and improve efficiency.
2.  `GridSearchCV` is run with 5-fold cross-validation, optimizing for the `f1_weighted` score.
3.  The best estimator is saved, and its performance is evaluated on the test set.
4.  Metrics like F1-score, accuracy, precision, recall, and AUC-ROC are calculated and stored.
5.  A detailed classification report is generated.

In [5]:
best_models = {}
model_performance_summary = []
all_classification_reports = ""
svm_grid_search_results = None # To store SVM grid results

model_list = [
    ("Logistic Regression", LogisticRegression), 
    ("K-Nearest Neighbors", KNeighborsClassifier), 
    ("Decision Tree", DecisionTreeClassifier), 
    ("Support Vector Machine", SVC),
    ("Naive Bayes", GaussianNB),
    ("Random Forest", RandomForestClassifier),
    ("XGBoost", XGBClassifier),
    ("ANN (PyTorch)", NeuralNetClassifier)
]

INPUT_DIM = X_train_scaled.shape[1]

for name, model_class in model_list:
    print(f"\n--- Running GridSearchCV for {name} ---")
    
    if name == "Support Vector Machine":
        model = model_class(random_state=42)
    elif name in ["K-Nearest Neighbors", "Naive Bayes"]:
        model = model_class()
    elif name == "Random Forest":
        model = model_class(random_state=42, n_jobs=-1)
    elif name == "XGBoost":
        model = model_class(random_state=42, use_label_encoder=False, eval_metric='logloss')
    elif name == "ANN (PyTorch)":
        # skorch wrapper for PyTorch model
        model = NeuralNetClassifier(
            module=MLP,
            module__in_dim=INPUT_DIM,
            criterion=nn.BCEWithLogitsLoss,
            optimizer=torch.optim.Adam,
            device='cuda' if torch.cuda.is_available() else 'cpu',
            verbose=0, # Suppress skorch's verbosity
            callbacks=[('early_stop', skorch.callbacks.EarlyStopping(patience=40, monitor='valid_loss'))]
        )
    else:
        model = model_class(random_state=42)

    grid_search = GridSearchCV(estimator=model, 
                               param_grid=param_grids[name], 
                               cv=5, 
                               scoring='f1_weighted', 
                               n_jobs=-1, 
                               verbose=1,
                               refit=True) 
    
    grid_search.fit(X_train_scaled, y_train)
    
    best_models[name] = grid_search.best_estimator_
    
    if name == "Support Vector Machine":
        svm_grid_search_results = grid_search
    
    print(f"Best Parameters found for {name}: {grid_search.best_params_}")
    
    y_pred = grid_search.best_estimator_.predict(X_test_scaled)
    
    if hasattr(grid_search.best_estimator_, "predict_proba"):
        y_pred_proba_raw = grid_search.best_estimator_.predict_proba(X_test_scaled)
        if name == "ANN (PyTorch)":
            y_pred_proba = y_pred_proba_raw[:, 1]
        else:
            y_pred_proba = y_pred_proba_raw[:, 1]
    else:
        y_pred_proba = y_pred 
    
    y_test_int = y_test.astype(int)
    y_pred_int = y_pred.astype(int)
    
    test_f1 = f1_score(y_test_int, y_pred_int, average='weighted')
    test_accuracy = accuracy_score(y_test_int, y_pred_int)
    test_precision = precision_score(y_test_int, y_pred_int, average='weighted')
    test_recall = recall_score(y_test_int, y_pred_int, average='weighted')
    test_auc = roc_auc_score(y_test, y_pred_proba)

    model_performance_summary.append({
        "Model": name,
        "F1-Score": test_f1,
        "Accuracy": test_accuracy,
        "Precision": test_precision,
        "Recall": test_recall,
        "AUC-ROC": test_auc,
        "Best Params": str(grid_search.best_params_)
    })

    report = classification_report(y_test_int, y_pred_int, target_names=['Healthy', 'Parkinsons'])
    all_classification_reports += f"--- Classification Report for {name} ---\n"
    all_classification_reports += f"Best Parameters: {grid_search.best_params_}\n"
    all_classification_reports += report + "\n" + "="*60 + "\n\n"


--- Running GridSearchCV for Logistic Regression ---
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best Parameters found for Logistic Regression: {'C': 1, 'class_weight': None, 'max_iter': 4000, 'penalty': 'l1', 'solver': 'saga'}

--- Running GridSearchCV for K-Nearest Neighbors ---
Fitting 5 folds for each of 80 candidates, totalling 400 fits
Best Parameters found for Logistic Regression: {'C': 1, 'class_weight': None, 'max_iter': 4000, 'penalty': 'l1', 'solver': 'saga'}

--- Running GridSearchCV for K-Nearest Neighbors ---
Fitting 5 folds for each of 80 candidates, totalling 400 fits
Best Parameters found for K-Nearest Neighbors: {'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'uniform'}
Best Parameters found for K-Nearest Neighbors: {'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'uniform'}

--- Running GridSearchCV for Decision Tree ---
Fitting 5 folds for each of 144 candidates, totalling 720 fits

--- Running GridSearchCV for Decision Tree ---
Fitting 5

c:\Users\madha\anaconda3\Lib\site-packages\xgboost\training.py:199: UserWarning: [08:32:11] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Best Parameters found for XGBoost: {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.7}

--- Running GridSearchCV for ANN (PyTorch) ---
Fitting 5 folds for each of 144 candidates, totalling 720 fits
Best Parameters found for ANN (PyTorch): {'batch_size': 32, 'lr': 0.01, 'max_epochs': 100, 'module__dropout': 0.1, 'module__hidden': (256, 128), 'optimizer__weight_decay': 1e-05}
Best Parameters found for ANN (PyTorch): {'batch_size': 32, 'lr': 0.01, 'max_epochs': 100, 'module__dropout': 0.1, 'module__hidden': (256, 128), 'optimizer__weight_decay': 1e-05}


### 5. Save and Display Results
The performance metrics from all models are compiled into a pandas DataFrame, sorted by F1-score, and saved to a CSV file. The detailed classification reports are saved to a text file. A summary of the performance is also printed to the console.

In [6]:
summary_df = pd.DataFrame(model_performance_summary).sort_values(by="F1-Score", ascending=False)
summary_csv_path = os.path.join(output_dir, 'model_performance_summary.csv')
summary_df.to_csv(summary_csv_path, index=False)
print(f"\nModel performance summary saved to: {summary_csv_path}")

reports_txt_path = os.path.join(output_dir, 'classification_reports.txt')
with open(reports_txt_path, 'w') as f:
    f.write(all_classification_reports)
print(f"Detailed classification reports saved to: {reports_txt_path}")

print("\n--- Final Model Performance After GridSearchCV ---")
print(summary_df.drop(columns='Best Params').to_string())


Model performance summary saved to: parkinsons_analysis_results_gridsearch\model_performance_summary.csv
Detailed classification reports saved to: parkinsons_analysis_results_gridsearch\classification_reports.txt

--- Final Model Performance After GridSearchCV ---
                    Model  F1-Score  Accuracy  Precision    Recall   AUC-ROC
5           Random Forest  0.939581  0.938776   0.941348  0.938776  0.971847
6                 XGBoost  0.901472  0.897959   0.912485  0.897959  0.968468
3  Support Vector Machine  0.873596  0.877551   0.873574  0.877551  0.914414
1     K-Nearest Neighbors  0.836735  0.836735   0.836735  0.836735  0.779279
7           ANN (PyTorch)  0.831462  0.836735   0.829723  0.836735  0.909910
0     Logistic Regression  0.822650  0.816327   0.835414  0.816327  0.898649
2           Decision Tree  0.822650  0.816327   0.835414  0.816327  0.837838
4             Naive Bayes  0.657465  0.632653   0.783633  0.632653  0.760135


### 6. Generate and Save Figures
This section creates several key visualizations to interpret the results:
- **Model Performance Comparison**: A bar plot showing the F1-scores of all tuned models.
- **Confusion Matrix**: A heatmap for the best-performing model, showing true vs. predicted labels.
- **ROC Curve**: A combined plot comparing the ROC curves and AUC scores for the top 4 models.
- **SVM Heatmap**: A heatmap visualizing the performance of the SVM with an RBF kernel across different `C` and `gamma` values.

In [7]:
print("\nGenerating Figures...")

# Model Performance Comparison
plt.figure(figsize=(12, 8))
sns.barplot(x='F1-Score', y='Model', data=summary_df, palette='viridis', orient='h', hue='Model', legend=False)
plt.title('Comparison of Tuned Model Performance (Test Set F1-Score)', fontsize=16)
plt.xlabel('Weighted F1-Score', fontsize=12)
plt.ylabel('Model', fontsize=12)
plt.xlim(min(0.7, summary_df['F1-Score'].min() * 0.95), 1.0)
fig_path = os.path.join(output_dir, 'model_comparison.png')
plt.savefig(fig_path, bbox_inches='tight')
plt.close()
print(f"Model Comparison plot saved to: {fig_path}")

# Confusion Matrix for the best model
try:
    print("Generating Confusion Matrix for best model...")
    best_model_name = summary_df.iloc[0]['Model']
    best_model = best_models[best_model_name]
    y_pred_best = best_model.predict(X_test_scaled)
    
    cm = confusion_matrix(y_test.astype(int), y_pred_best.astype(int))
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Healthy', 'Parkinsons'], 
                yticklabels=['Healthy', 'Parkinsons'])
    plt.title(f'Confusion Matrix for Best Model ({best_model_name})', fontsize=16)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.ylabel('True Label', fontsize=12)
    
    fig_path = os.path.join(output_dir, f'confusion_matrix_best_model.png')
    plt.savefig(fig_path, bbox_inches='tight')
    plt.close()
    print(f"Confusion Matrix saved to: {fig_path}")
except Exception as e:
    print(f"Could not generate Confusion Matrix: {e}")

# ROC Curve Plot
try:
    print("Generating Combined ROC Curve Plot...")
    plt.figure(figsize=(10, 8))
    plt.plot([0, 1], [0, 1], 'r--', label='Random Guess (AUC = 0.5)')
    
    top_4_model_names = summary_df.head(4)['Model'].tolist()
    
    for model_name in top_4_model_names:
        model = best_models[model_name]
        
        if hasattr(model, "predict_proba"):
            y_prob_raw = model.predict_proba(X_test_scaled)
            if len(y_prob_raw.shape) == 2:
                y_prob = y_prob_raw[:, 1]
            else:
                y_prob = y_prob_raw
            
            fpr, tpr, _ = roc_curve(y_test, y_prob)
            auc_score = roc_auc_score(y_test, y_prob)
            
            plt.plot(fpr, tpr, label=f'{model_name} (AUC = {auc_score:.4f})', lw=2)
        else:
            print(f"Skipping ROC for {model_name} (no predict_proba method)")

    plt.title('ROC Curve Comparison for Top Models', fontsize=16)
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate (Recall)', fontsize=12)
    plt.legend(loc='lower right')
    plt.grid(True, linestyle='--', alpha=0.6)
    
    fig_path = os.path.join(output_dir, 'roc_curve_comparison.png')
    plt.savefig(fig_path, bbox_inches='tight')
    plt.close()
    print(f"ROC Curve Plot saved to: {fig_path}")
except Exception as e:
    print(f"Could not generate ROC Curve Plot: {e}")

# SVM Heatmap
if svm_grid_search_results:
    try:
        cv_results = pd.DataFrame(svm_grid_search_results.cv_results_)
        rbf_results = cv_results[cv_results['param_kernel'] == 'rbf'].copy()
        
        if not rbf_results.empty:
            rbf_results = rbf_results[pd.to_numeric(rbf_results['param_gamma'], errors='coerce').notna()]
            rbf_results['param_C'] = pd.to_numeric(rbf_results['param_C'])
            rbf_results['param_gamma'] = pd.to_numeric(rbf_results['param_gamma'])
            
            pivoted_results = rbf_results.pivot(index='param_C', columns='param_gamma', values='mean_test_score')
            pivoted_results = pivoted_results.astype(float)

            plt.figure(figsize=(14, 9))
            sns.heatmap(pivoted_results, annot=True, fmt=".4f", cmap='plasma', linewidths=.5)
            plt.title('SVM (RBF Kernel) Performance Heatmap (F1-Score)', fontsize=16)
            plt.xlabel('Gamma (γ)', fontsize=12)
            plt.ylabel('Regularization (C)', fontsize=12)
            fig_path = os.path.join(output_dir, 'svm_heatmap.png')
            plt.savefig(fig_path, bbox_inches='tight')
            plt.close()
            print(f"SVM Heatmap saved to: {fig_path}")
    except Exception as e:
        print(f"Could not generate SVM heatmap: {e}")


Generating Figures...
Model Comparison plot saved to: parkinsons_analysis_results_gridsearch\model_comparison.png
Generating Confusion Matrix for best model...
Confusion Matrix saved to: parkinsons_analysis_results_gridsearch\confusion_matrix_best_model.png
Generating Combined ROC Curve Plot...
Model Comparison plot saved to: parkinsons_analysis_results_gridsearch\model_comparison.png
Generating Confusion Matrix for best model...
Confusion Matrix saved to: parkinsons_analysis_results_gridsearch\confusion_matrix_best_model.png
Generating Combined ROC Curve Plot...
ROC Curve Plot saved to: parkinsons_analysis_results_gridsearch\roc_curve_comparison.png
SVM Heatmap saved to: parkinsons_analysis_results_gridsearch\svm_heatmap.png
ROC Curve Plot saved to: parkinsons_analysis_results_gridsearch\roc_curve_comparison.png
SVM Heatmap saved to: parkinsons_analysis_results_gridsearch\svm_heatmap.png


### 7. Model Interpretability
To understand *why* the models are making their predictions, this section generates feature importance plots.
- For linear models (Logistic Regression, linear SVM), the coefficients are used.
- For tree-based models (Decision Tree, Random Forest), the built-in `feature_importances_` attribute is used.
- For non-linear "black box" models (KNN, RBF SVM, ANN), **Permutation Importance** is calculated. This method measures how much the model's performance decreases when a single feature's values are randomly shuffled, indicating its importance.

In [8]:
print("\n--- Generating Model Interpretability Plots ---")

# Logistic Regression Feature Importance
try:
    print("Generating Feature Importance for Logistic Regression...")
    log_reg_model = best_models['Logistic Regression']
    if hasattr(log_reg_model, 'coef_'):
        log_reg_importance = pd.DataFrame({
            'Feature': X.columns,
            'Importance': log_reg_model.coef_[0]
        }).sort_values(by='Importance', ascending=False)

        plt.figure(figsize=(12, 8))
        sns.barplot(x='Importance', y='Feature', data=log_reg_importance, palette='coolwarm', hue='Feature', legend=False)
        plt.title('Feature Importance (Coefficients) from Logistic Regression', fontsize=16)
        plt.xlabel('Coefficient Value', fontsize=12)
        plt.ylabel('Feature', fontsize=12)
        fig_path = os.path.join(output_dir, 'feature_importance_logreg.png')
        plt.savefig(fig_path, bbox_inches='tight')
        plt.close()
        print(f"Logistic Regression Importance plot saved to: {fig_path}")
except Exception as e:
    print(f"Could not generate Logistic Regression importance plot: {e}")

# Naive Bayes Feature Importance
try:
    print("Generating Feature Importance for Naive Bayes...")
    nb_model = best_models['Naive Bayes']
    nb_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance (Mean Diff)': np.abs(nb_model.theta_[1] - nb_model.theta_[0])
    }).sort_values(by='Importance (Mean Diff)', ascending=False)

    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance (Mean Diff)', y='Feature', data=nb_importance.head(10), palette='muted', hue='Feature', legend=False)
    plt.title('Top 10 Feature Importance (Absolute Mean Difference) from Naive Bayes', fontsize=16)
    plt.xlabel('Absolute Difference in Feature Means', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    fig_path = os.path.join(output_dir, 'feature_importance_naive_bayes.png')
    plt.savefig(fig_path, bbox_inches='tight')
    plt.close()
    print(f"Naive Bayes Importance plot saved to: {fig_path}")
except Exception as e:
    print(f"Could not generate Naive Bayes importance plot: {e}")

# Permutation Importance for KNN
try:
    print("Generating Permutation Importance for K-Nearest Neighbors...")
    knn_model = best_models['K-Nearest Neighbors']
    perm_importance_knn = permutation_importance(
        knn_model, X_test_scaled, y_test, n_repeats=10, random_state=42, n_jobs=-1
    )
    knn_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': perm_importance_knn.importances_mean
    }).sort_values(by='Importance', ascending=False)

    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=knn_importance.head(10), palette='plasma', hue='Feature', legend=False)
    plt.title('Top 10 Feature Importance (Permutation) from KNN', fontsize=16)
    plt.xlabel('Permutation Importance Mean', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    fig_path = os.path.join(output_dir, 'feature_importance_knn.png')
    plt.savefig(fig_path, bbox_inches='tight')
    plt.close()
    print(f"KNN Permutation Importance plot saved to: {fig_path}")
except Exception as e:
    print(f"Could not generate KNN importance plot: {e}")

# Permutation Importance for SVM
try:
    print("Generating Permutation Importance for Support Vector Machine...")
    svm_model = best_models['Support Vector Machine']
    
    if svm_model.kernel == 'linear':
        svm_importance = pd.DataFrame({
            'Feature': X.columns,
            'Importance': svm_model.coef_[0]
        }).sort_values(by='Importance', ascending=False)
        title = 'Feature Importance (Coefficients) from Linear SVM'
        xlabel = 'Coefficient Value'
    else:
        perm_importance_svm = permutation_importance(
            svm_model, X_test_scaled, y_test, n_repeats=10, random_state=42, n_jobs=-1
        )
        svm_importance = pd.DataFrame({
            'Feature': X.columns,
            'Importance': perm_importance_svm.importances_mean
        }).sort_values(by='Importance', ascending=False)
        title = f'Top 10 Feature Importance (Permutation) from SVM ({svm_model.kernel} kernel)'
        xlabel = 'Permutation Importance Mean'

    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=svm_importance.head(10), palette='viridis', hue='Feature', legend=False)
    plt.title(title, fontsize=16)
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    fig_path = os.path.join(output_dir, 'feature_importance_svm.png')
    plt.savefig(fig_path, bbox_inches='tight')
    plt.close()
    print(f"SVM Importance plot saved to: {fig_path}")
except Exception as e:
    print(f"Could not generate SVM importance plot: {e}")

# Decision Tree and Random Forest Feature Importance
try:
    print("Generating Feature Importance for Decision Tree...")
    best_dt = best_models['Decision Tree']
    importances_dt = pd.DataFrame({
        'Feature': X.columns,
        'Importance': best_dt.feature_importances_
    }).sort_values(by='Importance', ascending=False).head(10)

    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=importances_dt, palette='rocket', hue='Feature', legend=False)
    plt.title('Top 10 Feature Importances from Tuned Decision Tree', fontsize=16)
    plt.xlabel('Importance Score', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    fig_path = os.path.join(output_dir, 'feature_importance_dt.png')
    plt.savefig(fig_path, bbox_inches='tight')
    plt.close()
    print(f"Decision Tree Importance plot saved to: {fig_path}")

    print("Generating Feature Importance for Random Forest...")
    best_rf = best_models['Random Forest']
    importances_rf = pd.DataFrame({
        'Feature': X.columns,
        'Importance': best_rf.feature_importances_
    }).sort_values(by='Importance', ascending=False).head(10)

    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=importances_rf, palette='cividis', hue='Feature', legend=False)
    plt.title('Top 10 Feature Importances from Tuned Random Forest', fontsize=16)
    plt.xlabel('Importance Score', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    fig_path = os.path.join(output_dir, 'feature_importance_rf.png')
    plt.savefig(fig_path, bbox_inches='tight')
    plt.close()
    print(f"Random Forest Importance plot saved to: {fig_path}")

except Exception as e:
    print(f"Could not generate Tree Model importance plots: {e}")

# XGBoost Feature Importance
try:
    print("Generating Feature Importance for XGBoost...")
    best_xgb = best_models['XGBoost']
    importances_xgb = pd.DataFrame({
        'Feature': X.columns,
        'Importance': best_xgb.feature_importances_
    }).sort_values(by='Importance', ascending=False).head(10)

    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=importances_xgb, palette='magma', hue='Feature', legend=False)
    plt.title('Top 10 Feature Importances from Tuned XGBoost', fontsize=16)
    plt.xlabel('Importance Score', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    fig_path = os.path.join(output_dir, 'feature_importance_xgb.png')
    plt.savefig(fig_path, bbox_inches='tight')
    plt.close()
    print(f"XGBoost Importance plot saved to: {fig_path}")
except Exception as e:
    print(f"Could not generate XGBoost importance plot: {e}")

# Permutation Importance for ANN (PyTorch)
try:
    print("Generating Permutation Importance for ANN (PyTorch)...")
    ann_model = best_models['ANN (PyTorch)']
    
    perm_importance_ann = permutation_importance(
        ann_model, X_test_scaled, y_test, n_repeats=10, random_state=42, 
        n_jobs=1,  # Use 1 core to avoid potential pickling errors with complex objects
        scoring='f1_weighted'
    )
    ann_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': perm_importance_ann.importances_mean
    }).sort_values(by='Importance', ascending=False)

    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=ann_importance.head(10), palette='plasma', hue='Feature', legend=False)
    plt.title('Top 10 Feature Importance (Permutation) from ANN (PyTorch)', fontsize=16)
    plt.xlabel('Permutation Importance Mean (F1-Score Drop)', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    fig_path = os.path.join(output_dir, 'feature_importance_ann_pytorch.png')
    plt.savefig(fig_path, bbox_inches='tight')
    plt.close()
    print(f"ANN (PyTorch) Permutation Importance plot saved to: {fig_path}")
except Exception as e:
    print(f"Could not generate ANN (PyTorch) importance plot: {e}")


--- Generating Model Interpretability Plots ---
Generating Feature Importance for Logistic Regression...
Logistic Regression Importance plot saved to: parkinsons_analysis_results_gridsearch\feature_importance_logreg.png
Generating Feature Importance for Naive Bayes...
Logistic Regression Importance plot saved to: parkinsons_analysis_results_gridsearch\feature_importance_logreg.png
Generating Feature Importance for Naive Bayes...
Naive Bayes Importance plot saved to: parkinsons_analysis_results_gridsearch\feature_importance_naive_bayes.png
Generating Permutation Importance for K-Nearest Neighbors...
Naive Bayes Importance plot saved to: parkinsons_analysis_results_gridsearch\feature_importance_naive_bayes.png
Generating Permutation Importance for K-Nearest Neighbors...
KNN Permutation Importance plot saved to: parkinsons_analysis_results_gridsearch\feature_importance_knn.png
Generating Permutation Importance for Support Vector Machine...
KNN Permutation Importance plot saved to: parkin

### 8. Save Final Model
Finally, the best overall model (based on F1-score) and the `StandardScaler` are saved to disk using `joblib`. For the PyTorch model, `skorch` provides a `save_params` method to save the learned weights, as saving the entire model object can be problematic.

In [9]:
print("\n--- Saving Final Model and Scaler ---")
try:
    best_model_name = summary_df.iloc[0]['Model'] 
    final_model = best_models[best_model_name]
    model_filename = f'best_model_{best_model_name.replace(" ", "_")}.pkl'
    
    if best_model_name == 'ANN (PyTorch)':
        # For skorch models, save the parameters instead of the whole object
        final_model.save_params(f_params=f'{model_filename}_params.pkl')
        print(f"Final best model ({best_model_name}) PARAMS saved to {model_filename}_params.pkl")
    else:
        joblib.dump(final_model, model_filename)
        print(f"Final best model ({best_model_name}) saved to {model_filename}")

    joblib.dump(scaler, 'scaler.pkl')
    print("Scaler saved to scaler.pkl")
except Exception as e:
    print(f"Could not save final model: {e}")

print("\nAnalysis complete. All artifacts have been saved.")


--- Saving Final Model and Scaler ---
Final best model (Random Forest) saved to best_model_Random_Forest.pkl
Scaler saved to scaler.pkl

Analysis complete. All artifacts have been saved.
